# PandaSoup Scraper Tool

Parse data from websites into a pandas DataFrame for each page.

Start with a URL template, for example: "http://www.google.com?q1={}&q2={}&q3={}"

Each time we perform a scrape, we iterate over the list of params to plug into the template:
[
    ('hi', 'stuff', '123'),
    ('foo', '999', '2018/07/04'),
    ...
]

Each of these tuples creates a new URL, which we scrape with BeautifulSoup. Results are saved in the "raw_data" dictionary as values under its parameter tuple as the key.

Given an extract function (extract_func) to pull certain items out of each page of raw data to a dataframe, store in the list of dataframes. By default, the dict of pages is simply transformed to a dataframe with a 1-1 relationship, keeping the same key tuples.

User can give a "grouping strategy" to consolidate data based on parameters. 

Results can be saved to disk as a CSV file per DataFrame object.

Example: http://www.mywebsite.com?firstname={}&lastname={}&year={}&month={}&day={}, ["year", "month"]
* By default, data entities will be separated for each parameter combo: myfile_firstname_lastname_year_month_day.csv
* Using the strategy, they will be consolidated by year and month: myfile_year_month.csv

## Constructor args:
Pass a single dictionary. The following parameters are recognized.

    params_dict = {
        base : "http://www.mywebsite.com?q={}"
        param_names : [keyword1, keyword2, keyword3...]
        csv_base : "my/data/path/{}_{}_{}.csv"
        request_delay : 123
        extract_func: (lambda x: x)
        grouping_strategy : [keyword1, keyword2... ]
}


In [1]:
import math
import os
import pandas as pd
import re
import requests
import time

from __future__ import print_function
from bs4 import BeautifulSoup
from collections import defaultdict
from copy import deepcopy
# TODO: Add selenium functionality later

class PandaSoup:
    raw_data = defaultdict(list)
    request_delay = 5  # 5 seconds
    data = defaultdict(list)
    extract_func = (lambda x: x)
    grouping_strategy = None
    init_params = {}
    use_timestamped_csv = True
    
    def __init__(self, params_dict):
        self.base = params_dict.get('base', None)
        self.param_names = params_dict.get('param_names', None)
        self.grouping_strategy = params_dict.get('grouping_strategy', None)
        # If user specified a custom csv, don't use timestamp in file names
        if 'csv_base' in params_dict:
            self.use_timestamped_csv = False
            self.csv_base = params_dict['csv_base']
        elif 'grouping_strategy' in params_dict:
            self.csv_base = str.format("test/data/{}.csv", 
                                       "{}"+("_{}"*len(self.grouping_strategy)))
        else:
            self.csv_base = str.format("test/data/{}.csv", 
                                       "{}"+("_{}"*len(self.param_names)))
        self.request_delay = params_dict.get('request_delay', 5)
        self.extract_func = params_dict.get('extract_func', (lambda x: x))
        self.init_params = params_dict
        
    def debug_str(self, level=1):
        print("Raw data:", len(self.raw_data), "items")
        if level == 2:
            print("    keys:", self.raw_data.keys())
        print("Data:", len(self.data), "items")
        print("Default output path:", self.csv_base)
        if level == 2:
            print("    keys:", self.data.keys())
            print("Initialized with params:", self.init_params)
        
    def scrape(self, params=[], verbose=None):
        url = str.format(self.base, *params)
        if verbose:
            print("Reading", url)
        response = requests.get(url)
        return BeautifulSoup(response.text,"lxml")
        
    def scrape_all(self, params=[], reset=False, verbose=None):
        if reset:
            self.raw_data = {}
        for param_tuple in params:
            key = param_tuple
            if self.grouping_strategy:
                key = tuple(
                    [param_tuple[self.param_names.index(p)] for p in self.grouping_strategy]
                )
            if verbose:
                print("parameter values", param_tuple, "--> key", key)
            self.raw_data[key].append(self.scrape(param_tuple, verbose))
            time.sleep(self.request_delay)
        return self.raw_data
    
    def make_dataframes(self, reset=False, verbose=None):
        if reset:
            print("Clear dataframes")
            self.data = {}
        count = 0
        for key,value in self.raw_data.items():
            df = [self.extract_func(v) for v in value]
            self.data[key] = pd.concat(df)
            if verbose:
                print(key, "yielded", len(df), "rows of data")
        if verbose:
            print("Completed extracting data")
        return self.data
    
    def to_csv(self, verbose=None):
        files = []
        for key,value in self.data.items():
            file = self.csv_base 
            if self.use_timestamped_csv:
                file = str.format(self.csv_base, math.floor(time.time()*10), *key)
            else:
                file = str.format(self.csv_base, *key)
            if verbose:
                print("Writing to", file)
            value.to_csv(file)
            files.append(file)
        return files
        
    def reset_all(self,verbose=None):
        if verbose:
            print("Clearing data")
        self.raw_data = {}
        self.data = {}


In [5]:
print("You imported PandaSoup!")

You imported PandaSoup!


## Testing